In [2]:
# !pip install tensorflow --break-system-packages
# !pip install nltk --break-system-packages
# !pip install numpy --break-system-packages
# !pip install pandas --break-system-packages
# import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from keras.utils import to_categorical
import numpy as np
import nltk
import pandas as pd
import re

# Download the stopwords corpus if not already downloaded
nltk.download('stopwords')
nltk.download('reuters')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('webtext')
from nltk.corpus import stopwords
from nltk.corpus import reuters
from nltk.corpus import webtext
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer



2024-02-17 00:21:23.423308: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-17 00:21:23.441552: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-17 00:21:23.535981: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-17 00:21:23.536030: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-17 00:21:23.553224: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [3]:
def preprocess(content):
    
    words = word_tokenize(content)
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    filtered_words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words]
    # print(len(filtered_words))
    content = ' '.join(filtered_words)
    content = re.sub(r"[^a-zA-Z0-9. ]", "", content)
    content = re.sub(r'\s+', ' ', content)
    content = content.lower().strip()     
    return content

fileids = reuters.fileids()

# Initialize empty lists to store categories and raw text
categories = []
text = []
content = ""
# Loop through each file id and collect each files categories and raw text
for file in fileids:
    # categories.append(reuters.categories(file))
    # text.append(reuters.raw(file))
    content = content + reuters.raw(file)

content = preprocess(content)
# print(len(content))

In [4]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity

def euclidean(vec1,vec2):
    return np.linalg.norm(vec1 - vec2)

In [5]:
##Replace with original corpus
#
# corpus = [content[:500000]]

with open('GloVe/text8', 'r') as f:
    content = f.read().lower()

# content=content[:100000]
# print(content[:1000])
sentences=[content.split(" ")[:50000]]
corpus = sentences
# print(corpus)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
vocab_size = len(tokenizer.word_index) + 1

unique_words = list(tokenizer.word_index.keys())
# print(unique_words)

embed_size=100
window_size = 2

# print(sequences)

print("Vocab size is - ", vocab_size)

##Generate context,target pairs
context=[]
target=[]

# print(sequences)
context.append([0,0,0,0])
context.append([0,0,0,0])
target.append(sequences[0][0])
target.append(sequences[0][1])
for i in range(len(sequences)):

  for j in range(2,len(sequences[i])-2):
    new_cont = []
    for k in range(j-2,j+3):
      if k!=j:
        new_cont.append(sequences[i][k])

    context.append(new_cont)
    target.append(sequences[i][j])

target.append(sequences[0][-2])
target.append(sequences[0][-1])
context.append([0,0,0,0])
context.append([0,0,0,0])

# print(len(context),len(target))
context = np.array(context)
target = np.array(target)

# print(context)


Vocab size is -  7794


In [6]:
##build model

cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,)))
cbow.add(Dense(vocab_size, activation='softmax'))
cbow.compile(loss='categorical_crossentropy', optimizer='rmsprop')

print(cbow.summary())

one_hot = []

for target_val in target:
  one_hot.append(to_categorical(target_val,vocab_size))

one_hot=np.array(one_hot)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            779400    
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 7794)              787194    
                                                                 
Total params: 1566594 (5.98 MB)
Trainable params: 1566594 (5.98 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2024-02-17 00:22:05.232531: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-17 00:22:05.233117: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


None


In [7]:
cbow.fit(context,one_hot,epochs=100,batch_size=32)

cbow.save_weights('model_weights.h5')

weights = cbow.get_weights()[0]

print(weights)



Epoch 1/100
1563/1563 [==============================] - 5s 3ms/step - loss: 7.9509
Epoch 2/100
1563/1563 [==============================] - 5s 3ms/step - loss: 7.1196
Epoch 3/100
1563/1563 [==============================] - 4s 3ms/step - loss: 6.9415
Epoch 4/100
1563/1563 [==============================] - 4s 3ms/step - loss: 6.8402
Epoch 5/100
1563/1563 [==============================] - 5s 3ms/step - loss: 6.7611
Epoch 6/100
1563/1563 [==============================] - 5s 3ms/step - loss: 6.6898
Epoch 7/100
1563/1563 [==============================] - 5s 3ms/step - loss: 6.6254
Epoch 8/100
1563/1563 [==============================] - 4s 3ms/step - loss: 6.5634
Epoch 9/100
1563/1563 [==============================] - 4s 3ms/step - loss: 6.5046
Epoch 10/100
1563/1563 [==============================] - 4s 3ms/step - loss: 6.4434
Epoch 11/100
1563/1563 [==============================] - 4s 3ms/step - loss: 6.3852
Epoch 12/100
1563/1563 [==============================] - 4s 3ms/step - lo

In [8]:

words = ['japan','conflict','electronics','rift','trouble','fear']

for word in words:
    if word in unique_words:
        print("Similar words for ",word)
        word_index = tokenizer.word_index[word]
        word_vector = weights[word_index]
        similar_words = []
        for i in range(1,vocab_size):
            if i!=word_index:
                similar_words.append((unique_words[i-1],euclidean(word_vector,weights[i])))
                
        similar_words.sort(key=lambda x: x[1])
        print(similar_words[:5])

with open("vocab.txt","w") as f2:
    for item in unique_words:
        f2.write("%s\n" % item)

Similar words for  conflict
[('grief', 5.3781104), ('morality', 5.653295), ('blood', 5.663), ('struggle', 5.704911), ('fire', 5.7084312)]
Similar words for  electronics
[('biochemistry', 1.0752002), ('nutrition', 1.277063), ('vital', 1.3002056), ('vitamin', 1.3947284), ('calendars', 1.5490401)]
Similar words for  trouble
[('suffered', 5.327976), ('historians', 5.431294), ('originals', 5.727382), ('field', 5.7888293), ('difficulty', 5.790894)]
Similar words for  fear
[('authorization', 1.8831652), ('byzantine', 2.1199095), ('waldo', 2.2488813), ('campaigns', 2.3174365), ('grateful', 2.3740568)]
